In [1]:
import numpy as np
from scipy.optimize import newton_krylov

In [2]:
x1 = -4
y1 = 30
x2 = 0
y2 = 2
x3 = 4
y3 = 6

In [3]:
d = np.array([y1, y2, y3])
a = np.array([x1**2, x2**2, x3**2])
b = np.array([x1, x2, x3])
c = np.array([1, 1, 1])

Aprime = np.array([a, b, c]) #because of vector notation here I'm calling this array A'

Aprime

array([[16,  0, 16],
       [-4,  0,  4],
       [ 1,  1,  1]])

## Projection of d, data vector onto model

Model has the form $ y_i = \alpha(x_i)^2 + \beta(x_i) + \gamma + e_i $ for $i = 1,2,3
$

$\vec{a} = \begin{pmatrix} x_1^2 \\ x_2^2 \\ x_3^2 \end{pmatrix}$      $\vec{b} = \begin{pmatrix} x_1 \\ x_2 \\ x_3 \end{pmatrix}$   $\vec{c} = \begin{pmatrix} 1 \\ 1 \\ 1 \end{pmatrix}$

where $\vec{e}$ is an error matrix

first, project $\vec{d}$ onto model to solve for coefficients $\alpha ,\beta, \gamma$ held in the vector $\vec{x} = \begin{pmatrix} \alpha \\ \beta \\ \gamma \end{pmatrix}$

$\vec{x} = (AA')^{-1} \cdot A' \cdot \vec{d}$



In [4]:
#projection of d onto a to solve for a, b, c

A = np.transpose(Aprime) #the way A matrix was initialized made it really A transpose
#this will return A by taking the transpose of A'

dot = Aprime @ d #this returns a vector with three elements (3x1)

AA = Aprime @ A #this returns a 3x3 matrix

inverse = np.linalg.inv(AA) #returns the inverse of A times A' matrix also a 3x3 matrix

x = inverse @ dot #returns a vector with three elements 3x1


In [5]:
alpha = x[0]
beta = x[1]
gamma = x[2]

In [6]:
print('alpha is:', alpha, 
      'beta is :', beta, 
      'gamma is :',gamma)

alpha is: 1.0 beta is : -3.0 gamma is : 2.0


The resulting values for $\alpha, \beta, \gamma$ are consistent with the values a,b,c from part a, where we solved the system of equations by hand 

## solving for $\vec{e}$ error vector

$\vec{e} = \vec{d} - P \vec{d}$

where $P = A (A'A)^{-1} A' $

In [8]:
P = A @ inverse @ Aprime

In [9]:
e = d - P @ d

In [10]:
e

array([0., 0., 0.])

the error vector is $\vec{e} = \begin{pmatrix} 0 \\ 0 \\ 0 \end{pmatrix}$

### suppose $y_i = A(x_i - B)(x_i -C)$ for $i = 1,2,3$

Write a system of nonlinear equations for unknowns A,B,C

$ 30 = A(-4-B)(-4-C) $      
$ 2 = ABC $         
$ 6 = A(4-B)(4-C) $


Solve the resulting system of equations using Newton's Method
Use A=1, B=2, C=3 as the initial state



In [11]:
#define array for the system of equations F(x)

def F(x):
    A, B, C = x
    return np.array([A * (-4-B) * (-4-C) - 30, 
                    A * B * C - 2, 
                    A * (4-B) * (4-C) - 6])

#define the jacobian matrix of F(x)

def Jac(x):
    A, B, C = x
    return np.array([
        [(-4-B)*(-4-C), A*(C+4), A*(B+4)],
        [B*C, -A*C, -A*B], 
        [(4-B)*(4-C), A*(C-4), A*(B-4)]
        ])

#define initial guess for the solution
A=1
B=2
C=3

x0 = np.array([A,B,C])

In [15]:
#using Newton's Method, find the solution

def newton(f, Df, x0, epsilon=1e-6, max_iter = 100):
    xn = x0
    for n in range(0,max_iter):
        fxn = f(xn)
        if abs(fxn) < epsilon:
            print('Found solution after', n, 'iterations.')
            return xn
        Dfxn = Df(xn)
        if Dfsn == 0:
            print('Zero derivative. No solution found.')
            return None
        xn = xn - fxn/Dfxn
    print('Exceeded maximum iterations. No solution found.')
    return None



In [16]:
solution = newton(F, Jac, x0, epsilon=1e-6, max_iter=100)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()